In [17]:
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
import skimage.feature.texture as ski
import math
from sklearn.neighbors import KNeighborsClassifier
import time

In [2]:
def get_paragraph(gray_img, bin_img):

    height, width = gray_img.shape

    contours, hierarchy = cv.findContours(bin_img, cv.RETR_LIST, cv.CHAIN_APPROX_SIMPLE)

    threshold_width = 1500


    up, down, left, right = 0, height - 1, 0, width - 1

    for cnt in contours:
        x, y, w, h = cv.boundingRect(cnt)

        if w < threshold_width:
            continue

        if y < height / 2:
            if y > up:
                up = y
        else:
            down = y

    th = 0
    bin_img = bin_img[up:down + 1, left:right + 1]
    gray_img = gray_img[up:down + 1, left:right + 1]
    # Apply erosion to remove noise and dots.
    kernel = np.ones((3, 3), np.uint8)
    bin_img = cv.erode(bin_img, kernel, iterations=3)
    pts = np.nonzero(bin_img)
    x_min, y_min, x_max, y_max = min(pts[0]), min(pts[1]), max(pts[0]), max(pts[1])
    bin_img = bin_img[x_min-th:x_max+th, y_min-th:y_max+th]
    gray_img = gray_img[x_min-th:x_max+th, y_min-th:y_max+th]
    # Return the handwritten paragraph
    return gray_img, bin_img

def preprocessing(gray_img):
    gray_img = cv.GaussianBlur(gray_img, (5, 5), 0)
    ## (2) threshold
    thresh, bin_img = cv.threshold(gray_img, 0, 255, cv.THRESH_BINARY_INV|cv.THRESH_OTSU)
    gray_img, bin_img = get_paragraph(gray_img, bin_img)
    thresh, bin_img = cv.threshold(gray_img, 0, 255, cv.THRESH_BINARY_INV|cv.THRESH_OTSU)
#     plt.imshow(bin_img)
    hist = cv.reduce(bin_img,1, cv.REDUCE_AVG).reshape(-1)
#     for i in range(len(hist)):
#         print(i, hist[i])
    th = 2
    H,W = bin_img.shape[:2]
    uppers = []
    lowers = []
    if hist[0] > th:
        uppers.append(0)
    
     
    for y in range(H-1):
        if hist[y]<=th and hist[y+1]>th:
            uppers.append(y)
     
    for y in range(H-1):
        if hist[y]>th and hist[y+1]<=th:
            lowers.append(y)
            
    if hist[len(hist)-1] > th:
        lowers.append(len(hist)-1)

    img = cv.cvtColor(gray_img, cv.COLOR_GRAY2BGR)
    
    lines = []
    temp_uppers = uppers.copy()
    temp_lowers = lowers.copy()
    for i in range(len(uppers)):
        if lowers[i] - uppers[i] > 50:
            lines.append(img[uppers[i]:lowers[i], :])
        else:
            temp_uppers.remove(uppers[i])
            temp_lowers.remove(lowers[i])
    #print(temp_uppers)
    #print(temp_lowers)
    count = 1
    for l in lines:
        #cv.imwrite("line" + str(count) + ".png", l)
        count+=1
    
    return lines

In [14]:
#get the neighbours in a list by moving clockwise
def getNeighbours(pointCoord):
    neighboursCoord = []
    '''
    get the neighbours starting from the top-left clockwise
    '''
    #print(pointCoord)
    #append the top neighbours moving from left to right
    for j in range(0,3):
        neighboursCoord.append([pointCoord[0] - 1,j])
    #append the right neighbour
    neighboursCoord.append([pointCoord[0],pointCoord[1] + 1])
    #append the bottom neighbours moving from right to left
    for j in range(2,-1,-1):
        neighboursCoord.append([pointCoord[0] + 1,j])
    #append the left neighbour
    neighboursCoord.append([pointCoord[0],pointCoord[1] - 1])
    return neighboursCoord


def CSP_LP(neighbours,N,Threshold):
    #get the accumlating of csp_lp from the neighbours
    CSP_LP_SUM = 0
    #loop on all neighbours
    for i in range(0,int(N/2)):
        #substract each neighbour and its peer
        s = neighbours[i] - neighbours[i + int(N/2)]
        #check if s is greater than the given threshold
        if(s > Threshold):
            #if the condition is satisfied then add pow(2,i) to csp_lp sum
            CSP_LP_SUM += (2**i)
    return CSP_LP_SUM

'''
Testing phase
'''
pixels = [
    [0.5,0.9,0.6],
    [0.8,-0.2,0.7],
    [0.10,0.88,0.5]
]
neighbours = []
neighboursCoord = getNeighbours([1,1])
for i in neighboursCoord:
    x = i[0]
    y = i[1]
    neighbours.append(pixels[x][y])
CSP_LP_SUM = CSP_LP(neighbours,8,0.01)
print(CSP_LP_SUM)
CSP_LP_Matrix = [
    [9,15,2,15],
    [9,10,2,6],
    [0,15,2,6],
    [9,15,2,6]
]
GLCM = ski.greycomatrix(CSP_LP_Matrix,distances=[2],angles=[0,math.pi/4,
                                                    math.pi/2,math.pi*(5/4)],levels=16)
#print(GLCM[:,:,0,3])
featureVector = []
for k in range(GLCM.shape[3]):
    for i in range(GLCM.shape[0]):
        for j in range(GLCM.shape[1]):
            featureVector.append(GLCM[i][j][0][k])
#print(len(featureVector))
#print(featureVector)

6


In [31]:
def getFeatureVector(img):
    GLCM_Input = []
    startTime = time.time()
    imgRows = img.shape[0]
    imgCols = img.shape[1]
    for i in range(1,imgRows - 1):
        GLCM_Row = []
        for j in range(1,imgCols - 1):
            neighbours = []
            neighboursCoord = getNeighbours([i,j])
            for p in neighboursCoord:
                x = p[0]
                y = p[1]
                neighbours.append(img[x][y])
            CSP_LP_SUM = CSP_LP(neighbours,8,0.01)
            GLCM_Row.append(CSP_LP_SUM)
        GLCM_Input.append(GLCM_Row)
    
    GLCM_Matrix = ski.greycomatrix(GLCM_Input,distances=[2],angles=[0,math.pi/4,
                                                    math.pi/2,math.pi*(5/4)],levels=16)
    #endTime = time.time()
    #print("time = ",startTime-endTime)
    featureVector = []
    for k in range(GLCM_Matrix.shape[3]):
        for i in range(GLCM_Matrix.shape[0]):
            for j in range(GLCM_Matrix.shape[1]):
                featureVector.append(GLCM_Matrix[i][j][0][k])
    endTime = time.time()
    print("time = ",endTime - startTime)
    return featureVector

In [36]:
#read the image and convert to normalized grayscale
#img = np.array(cv.imread("../formsE-H/e01-014.png",0))
img = cv.imread('../formsE-H/e01-014.png')
gray_img = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
#print(img)
#print(img.shape)
lines = preprocessing(gray_img)
features = []
labels = []

start = time.time()
for line in lines:
    startTime = time.time()
    line = cv.cvtColor(line, cv.COLOR_BGR2GRAY)/255
    endTime = time.time()
    print(endTime-startTime)
    print(line.shape)
    lineFeature = getFeatureVector(line)
    features.append(lineFeature)
end = time.time()
print(end-start)

kNeighbours = KNeighborsClassifier(n_neighbors=3)
#kNeighbours.fit(features,labels)
    
#plt.imshow(img)

0.0002868175506591797
(78, 1779)
time =  0.8004510402679443
0.0005078315734863281
(105, 1779)
time =  1.0901598930358887
0.0020494461059570312
(77, 1779)
time =  0.838660478591919
0.0005176067352294922
(89, 1779)
time =  1.1723558902740479
0.0007598400115966797
(90, 1779)
time =  1.2125370502471924
0.0005600452423095703
(82, 1779)
time =  0.8318030834197998
0.000499725341796875
(62, 1779)
time =  0.6157441139221191
6.570028305053711
